In [1]:
%load_ext bigdata
%hive_start
%timeout 300

In [2]:
!hdfs dfs -ls /tmp
!hdfs dfs -mkdir /tmp/wordcount
!hdfs dfs -ls /tmp/


Found 2 items
drwxrwx---   - root supergroup          0 2020-02-18 15:39 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 15:49 /tmp/hive
Found 3 items
drwxrwx---   - root supergroup          0 2020-02-18 15:39 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2020-02-18 15:49 /tmp/hive
drwxr-xr-x   - root supergroup          0 2020-02-18 15:52 /tmp/wordcount


In [5]:
!hdfs dfs -copyFromLocal data.tsv  /tmp/wordcount/

In [6]:
%%hive

DROP TABLE IF EXISTS data;
DROP TABLE IF EXISTS letters_counts;

DROP TABLE IF EXISTS data;
OK
Time taken: 31.336 seconds
DROP TABLE IF EXISTS letters_counts;
OK
Time taken: 0.018 seconds


In [8]:
%%hive

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;

CREATE TABLE data (letter STRING, dat STRING, numb INT)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY '\t';
OK
Time taken: 4.767 seconds
LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE data;
Loading data to table default.data
OK
Time taken: 3.722 seconds


In [12]:
%%hive
CREATE TABLE letters_counts AS
(SELECT letter, COUNT(*) FROM data GROUP BY letter);

CREATE TABLE letters_counts AS
(SELECT letter, COUNT(*) FROM data GROUP BY letter);
Query ID = root_20200218160807_abe1456f-9e20-4f28-910a-b21d53aef520
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1582040149898_0002, Tracking URL = http://900d27d24b85:8088/proxy/application_1582040149898_0002/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582040149898_0002
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2020-02-18 16:08:27,086 Stage-1 map = 0%,  reduce = 0%
2020-02-18 16:08:41,006 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 4.2 sec
2020-02-18 16:08:53,587 Stage-1 map = 1

In [14]:
%%hive
INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM letters_counts;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM letters_counts;
Query ID = root_20200218161006_0ef37922-04a3-4ebc-b904-f1159a417406
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks is set to 0 since there's no reduce operator
Starting Job = job_1582040149898_0003, Tracking URL = http://900d27d24b85:8088/proxy/application_1582040149898_0003/
Kill Command = /usr/local/hadoop/bin/hadoop job  -kill job_1582040149898_0003
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 0
2020-02-18 16:10:26,272 Stage-1 map = 0%,  reduce = 0%
2020-02-18 16:10:38,520 Stage-1 map = 100%,  reduce = 0%, Cumulative CPU 3.57 sec
MapReduce Total cumulative CPU time: 3 seconds 570 msec
Ended Job = job_1582040149898_0003
Stage-3 is selected by condition resolver.
Stage-2 is filtered out by condition resolver.
Stage-4 is filtered out by condition resolver.
Moving data to directory hdfs://0.0.0.0:9000/tmp/output/.hive-stagi

In [15]:
!hive -S -e 'source output.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
File: output.hql is not a file.


In [16]:
!hdfs dfs -cat /tmp/output/000000_0 | head

A,12
B,10
C,4
D,1
E,13


In [17]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [18]:
!cat output/000000_0

A,12
B,10
C,4
D,1
E,13
